## Project 4
## Submitted by Roshan Lulu

### Step 1 - Get the Job Posting data from the website

- Identify the Job titles to scrap
- Identify the element/child relationships and get the attributes
- Baby step - Get 1 job posting scraped


### A. IMPORT LIBRARIES

In [1]:
# import libraries
import scrapy
from scrapy.http import Request
from scrapy import Selector
import requests
import pandas as pd


### B. PARSE SEARCH RESULTS FOR THE JOB POST LINKS

In [2]:
# # deinfe the serach urls
# urls = [{'page':1,'type': 'data-analyst', 'url':'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
#                 {'page':1,'type': 'data-science', 'url':'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
#                  {'page':1, 'type': 'business-intelligence', 'url':'https://www.seek.com.au/Business-Intelligence-jobs/in-All-Sydney-NSW'}]

job_desc_i_want = ['Data-Science', 'Data-Analyst', 'Business-Intelligence']
where_is_the_job = '-jobs/in-All-Sydney-NSW'
search_urls = []

for index in range(0, len(job_desc_i_want)):
    job_dict = {}
    job_dict['type'] = job_desc_i_want[index]
    job_dict['page'] = 1
    job_dict['url'] = 'https://www.seek.com.au/' + job_desc_i_want[index] + where_is_the_job
    search_urls.append(job_dict)

search_urls

[{'page': 1,
  'type': 'Data-Science',
  'url': 'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
 {'page': 1,
  'type': 'Data-Analyst',
  'url': 'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
 {'page': 1,
  'type': 'Business-Intelligence',
  'url': 'https://www.seek.com.au/Business-Intelligence-jobs/in-All-Sydney-NSW'}]

In [3]:
pagelinks_df = pd.DataFrame(columns = ('Job_Desc', 'Page_Num', 'url_link'))

In [4]:
# Define the start request function
def get_all_page_links_per_job(url, job_desc, page_num=1):
    # Get the html response from the first page url
    response = requests.get(url)
    # get the link to the next page
    next_page_anchor  =  Selector(text=response.text).xpath("//a[@data-automation='page-next']/@href").extract()
    # check if there is a next page
    if len(next_page_anchor) > 0:
        # Save the current url details before finding the next one
        pagelinks_df.loc[len(pagelinks_df),:] = [job_desc, page_num, url]
        # Start searching for next page link
        # Recursively call to get the links in next anchor page
        page_num += 1
        next_page_anchor = "https://www.seek.com.au" + next_page_anchor[0]
        get_all_page_links_per_job(next_page_anchor, job_desc, page_num)
    else:
        pass

In [5]:
def parse_all_first_pages(urls=search_urls):
    for url in urls:
        get_all_page_links_per_job(url['url'], url['type']) 

#Parse all the required job search results and get the links to the job postings
parse_all_first_pages()

In [6]:
# Display the dataframe obtained
pagelinks_df.head()

,Job_Desc,Page_Num,url_link
0,Data-Science,1,https://www.seek.com.au/Data-Science-jobs/in-A...
1,Data-Science,2,https://www.seek.com.au/Data-Science-jobs/in-A...
2,Data-Science,3,https://www.seek.com.au/Data-Science-jobs/in-A...
3,Data-Science,4,https://www.seek.com.au/Data-Science-jobs/in-A...
4,Data-Science,5,https://www.seek.com.au/Data-Science-jobs/in-A...


In [7]:
# Get all the job posting links from each page
list_of_posts = []
for index in range(0, len(pagelinks_df)):
    # Get the response text from the url
    url = pagelinks_df.iloc[index, 2]
    response = requests.get(url)
    # Parse the response text
    # parse_job_posting(response)
    post = Selector(text=response.text).xpath('//a[@data-automation="jobTitle"]/@href').extract()
    list_of_posts.append(post)

pagelinks_df['Job_Posts'] = list_of_posts
pagelinks_df

,Job_Desc,Page_Num,url_link,Job_Posts
0,Data-Science,1,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33390778?type=standard&tier=no_tier&pos=...
1,Data-Science,2,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33344575?type=standout&tier=no_tier&pos=...
2,Data-Science,3,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33314605?type=standout&tier=no_tier&pos=...
3,Data-Science,4,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33306870?type=standard&tier=no_tier&pos=...
4,Data-Science,5,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33259310?type=standard&tier=no_tier&pos=...
5,Data-Science,6,https://www.seek.com.au/Data-Science-jobs/in-A...,[]
6,Data-Science,7,https://www.seek.com.au/Data-Science-jobs/in-A...,[]
7,Data-Science,8,https://www.seek.com.au/Data-Science-jobs/in-A...,[/job/33375729?type=standout&tier=no_tier&pos=...
8,Data-Analyst,1,https://www.seek.com.au/Data-Analyst-jobs/in-A...,[/job/33392671?type=standard&tier=no_tier&pos=...
9,Data-Analyst,2,https://www.seek.com.au/Data-Analyst-jobs/in-A...,[/job/33353361?type=standout&tier=no_tier&pos=...


### C. PARSE JOB POSTINGS FOR THE TEXT TAGS!

In [8]:
### C. Go through each page link and get the 
# Description, Job_Title, Salary, Location, Work_Type, Classification,
# DatePosted, Postedby

In [36]:
job_posts_df = pd.DataFrame(columns = ('url', 'Title', 'JD_1', 'JD_2'))

In [137]:
# Define the start request function

rows = []
for index in range(0, 1):
    # Get the response text from the url
    urls = pagelinks_df.iloc[index, 3]
    for url in urls:
        url = 'https://www.seek.com.au' + url
        response = requests.get(url)
        job_title = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div[@class="grid_6"]/h1/text()').extract_first()
        job_adv = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div[@class="grid_6"]/span/text()').extract_first()
        job_desc_1 = Selector(text=response.text).xpath('//div[@class="templatetext"]/p//text()').extract_first()
        job_desc_2 = Selector(text=response.text).xpath('//div[@class="templatetext"]/ul/li//text()').extract_first()
        job_pay = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div[@itemprop="baseSalary"]/text()').extract_first()
        job_type = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div[@itemprop="employmentType"]/text()').extract_first()
        job_class = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="industry"]/text()').extract_first()
        job_region = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="addressRegion"]//text()').extract_first()
        job_location = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="addressLocality"]//text()').extract_first()
        rows.append([url, job_title, job_desc_1, job_desc_2, job_adv, job_pay, job_type, job_class, job_region, job_location])

In [138]:
job_posts_df = pd.DataFrame(rows, columns = ('url', 'Title', 'JD_1', 'JD_2','Advertiser', 'Salary', 'Work Type', 'Classification', 'Region', 'Location'))

In [140]:
job_posts_df.head(10)


,url,Title,JD_1,JD_2,Advertiser,Salary,Work Type,Classification,Region,Location
0,https://www.seek.com.au/job/33390778?type=stan...,Senior Manager - Data Science,I am currently looking for a,You will possess a strong background in: Stati...,Finite IT Recruitment Solutions -,"$150 - $180,000 package",Full Time,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
1,https://www.seek.com.au/job/33349090?type=stan...,Head of Data Science - ML & AI,"We are looking for a Head of Data Science, Mac...","Strong experience in analytical tools, process...",Ethos BeathChapman -,None,Full Time,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
2,https://www.seek.com.au/job/33338994?type=stan...,Head of Data Science & Analytics | Strategic r...,None,A unique Career opportunity along with an exce...,Data Talent Pty Ltd -,Excellent Salary Package on offer,Full Time,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
3,https://www.seek.com.au/job/33338996?type=stan...,Head of Data Science - Unique Career Opportuni...,As head of Data Science will lead a team of da...,A unique Career opportunity along with an exce...,Data Talent Pty Ltd -,Excellent Salary Package plus Bonus,Full Time,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
4,https://www.seek.com.au/job/33329261?type=stan...,Head of Data Science &amp; Products,Our Company,"Provide leadership, mentoring and development ...",TABCORP -,None,Full Time,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
5,https://www.seek.com.au/job/33329262?type=stan...,Head of Data Science &amp; Products,Our Company,"Provide leadership, mentoring and development ...",TABCORP -,None,Full Time,Science & Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
6,https://www.seek.com.au/job/33329263?type=stan...,Head of Data Science &amp; Products,Our Company,"Provide leadership, mentoring and development ...",TABCORP -,None,Full Time,Science & Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
7,https://www.seek.com.au/job/33329506?type=stan...,Computational Statistician | Data Science and ...,About Us,Creating analysis and visualization features f...,Displayr -,None,Full Time,Science & Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
8,https://www.seek.com.au/job/33314604?type=stan...,"Junior Data Scientist (Modelling, Insights & D...",None,Engaging with internal stakeholders to consult...,Correlate Resources -,$60-90k,Full Time,Consulting & Strategy,[],[Sydney]
9,https://www.seek.com.au/job/33248755?type=stan...,DATA SCIENCE EXPERT,None,6-month contract,Opus Recruitment Solutions -,$800 - 1000 per annum,Contract/Temp,Information & Communication Technology,"[ CBD, Inner West & Eastern Suburbs]",[Sydney]
